**Import libraries**

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime

### **Scrapping currencies**

**Declaring currencies URL**

In [2]:
bitcoin = 'https://coinmarketcap.com/currencies/bitcoin/'
jupiter = 'https://coinmarketcap.com/currencies/jupiter-ag/'
tars = 'https://coinmarketcap.com/currencies/tars-protocol/'
solana = 'https://coinmarketcap.com/currencies/solana/'
ethereum = 'https://coinmarketcap.com/currencies/ethereum/'
xrp = 'https://coinmarketcap.com/currencies/xrp/'
kaspa = 'https://coinmarketcap.com/currencies/kaspa/'
notcoin = 'https://coinmarketcap.com/currencies/notcoin/'

# storage the links in a list

data = {
    "Bitcoin": bitcoin,
    "Jupiter": jupiter,
    "Tars Protocol": tars,
    "Solana": solana,
    "Ethereum": ethereum,
    "XRP": xrp,
    "Kaspa": kaspa,
    "Notcoin": notcoin
}

# convert the dictionary to a pandas DataFrame

links = pd.DataFrame(data.items(), columns=["Currency", "Link"])
links

,Currency,Link
0,Bitcoin,https://coinmarketcap.com/currencies/bitcoin/
1,Jupiter,https://coinmarketcap.com/currencies/jupiter-ag/
2,Tars Protocol,https://coinmarketcap.com/currencies/tars-prot...
3,Solana,https://coinmarketcap.com/currencies/solana/
4,Ethereum,https://coinmarketcap.com/currencies/ethereum/
5,XRP,https://coinmarketcap.com/currencies/xrp/
6,Kaspa,https://coinmarketcap.com/currencies/kaspa/
7,Notcoin,https://coinmarketcap.com/currencies/notcoin/


**Creating price dataframe**

In [3]:
price_df = pd.DataFrame(columns = ['Currency', 'Price', 'Market cap', 'Timestamp'])
price_df

,Currency,Price,Market cap,Timestamp


**Setting up execution time frames**

In [4]:
timeout = time.time() + 60*5

**Creating loop**

In [6]:
while True:
    c = 0
    
    # for loop to iterate links and scrapp values
    for index, row in links.iterrows():

        # request page
        response = requests.get(row['Link'])
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # find price
        price = soup.find('span', class_='sc-65e7f566-0 clvjgF base-text')
        # find market cap
        marketcap = soup.find('dd', class_='sc-65e7f566-0 dzgtSD base-text').text
        
        # marketcap cleaning
        dollar_index = marketcap.find('$')
        if dollar_index != -1:
            marketcap = marketcap[dollar_index:]

        # setting transition dataframe
        record = pd.DataFrame({
        'Currency': row['Currency'],
        'Price': price,
        'Market cap': marketcap,
        'Timestamp': datetime.today().strftime('%m/%d/%Y %H:%M')
        }, index=[0])

        #print(name)
        print(record)

        # record append on price_df
        price_df = pd.concat([price_df, record], ignore_index=True)
    
    # timeframe repeat
    time.sleep(150)

    # close loop
    if c == 5 or time.time() > timeout:
        break
    

  Currency       Price          Market cap         Timestamp
0  Bitcoin  $59,493.15  $1,175,192,780,383  08/20/2024 18:17
  Currency    Price      Market cap         Timestamp
0  Jupiter  $0.7886  $1,064,654,961  08/20/2024 18:17
        Currency    Price   Market cap         Timestamp
0  Tars Protocol  $0.1392  $17,260,802  08/20/2024 18:17
  Currency    Price       Market cap         Timestamp
0   Solana  $143.46  $66,912,791,126  08/20/2024 18:17
   Currency      Price        Market cap         Timestamp
0  Ethereum  $2,591.62  $311,743,217,201  08/20/2024 18:17
  Currency    Price       Market cap         Timestamp
0      XRP  $0.5967  $33,482,725,660  08/20/2024 18:17
  Currency    Price      Market cap         Timestamp
0    Kaspa  $0.1611  $3,946,728,959  08/20/2024 18:17
  Currency     Price      Market cap         Timestamp
0  Notcoin  $0.01069  $1,095,501,975  08/20/2024 18:17


In [ ]:
price_df